In [6]:
import pandas as pd
from glob import glob
import os
import xml.etree.ElementTree as ET
import xmltodict
import json
from lxml import etree
import codecs
import re
from extraction_data import *

In [7]:
path = "../data/MainMatterReport_usang_20150101_20230919.csv"
df = pd.read_csv(path, encoding='utf-8-sig')
df = df[(df['parser 에러여부']==True) 
        # & (df['ic_mthn'].isin(["주주배정증자"]))
        ].copy()
df = df.sort_values(['접수일'], ascending=False)

print(len(df), df.columns)

594 Index(['rcept_no', 'corp_cls', 'corp_code', 'corp_name', 'nstk_ostk_cnt',
       'nstk_estk_cnt', 'fv_ps', 'bfic_tisstk_ostk', 'bfic_tisstk_estk',
       'fdpp_fclt', 'fdpp_bsninh', 'fdpp_op', 'fdpp_dtrp', 'fdpp_ocsa',
       'fdpp_etc', 'ic_mthn', 'ssl_at', 'ssl_bgd', 'ssl_edd', '접수일', '보고서유무',
       'parser 에러여부', '보고서 종류'],
      dtype='object')


In [ ]:
ic_mthn_df = df[df['ic_mthn'].isin(["주주배정후 실권주 일반공모", "주주배정증자"])].copy()


In [9]:
## test용 코드
save_path = "../data/doc/"
rcept_no = "20230914000136"

xml_file_path = os.path.join(save_path, "%s.xml"%rcept_no)
# XML 파일 읽기
with open(xml_file_path, "r", encoding="utf-8-sig") as xml_file:
    # XML을 딕셔너리로 변환
    xml_dict = xmltodict.parse(xml_file.read())
    

# 시작 마커와 끝 마커 사이의 텍스트 추출
start_markers = ["신주인수권증서에 관한 사항", "신주인수권에 관한 사항",]
end_markers = ["(마) 기타 참고사항", "바.  기타 참고사항", "마. 기타 참고사항", "마. 청약이 금지되는 공매도 거래 기간"]
try:
    NST_RM = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['LIBRARY']['P']
except TypeError as e:
    NST_RM = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["P"]

NST_RM = text_extranc(NST_RM, start_markers, end_markers, printer=False)

if NST_RM in ["", "-"]:
    NST_RM = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['LIBRARY']['P']
    end_markers = None
    NST_RM = text_extranc(NST_RM, start_markers, end_markers, printer=False)
NST_RM

': 신주인수권증서 미발행9) 기타사항① 청약증거금은 납입일에 주금 납입금으로 대체하며 청약일로부터 납입일까지의 이자는 없습니다.② 1주 미만의 단수주는 절사하여 배정하지 않습니다.③ 상기의 기재사항은 관계기관과의 협의 및 진행과정에서 변경될 수 있습니다.④ 기타 본건 신주발행과 관련한 부수적인 사항은 대표집행임원에게 위임합니다.'

In [10]:

columns = ["RCEPT_NO",
           "CST_CNT", # 신주의 종류와 수에서 보통주식(주)의 가격
           "PST_CNT", # 신주의 종류와 수에서 기타주식(주)의 가격
           "CST_ISS_VAL", # 신주 발행가액에서 확정발행가의 보통주식(주)의 가격
           "PST_ISS_VAL", # 신주 발행가액에서 확정발행가의 기타주식(주)의 가격
           "PRE_CST_VAL", # 신주 발행가액에서 예정발행가의 보통주식(주)의 가격
           "PRE_PST_VAL", # 신주 발행가액에서 예정발행가의 기타주식(주)의 가격
           "PRE_CST_VAL_DT", # 신주 발행가액에서 예정발행가의 보통주식(주)의 확정예정일
           "PRE_PST_VAL_DT", # 신주 발행가액에서 예정발행가의 기타주식(주)의 확정예정일
           "ISS_FUNC", # 신주 발행가 산정방법
           "ALL_BS_DT", # 신주배정기준일
           "NEW_ASN_CNT", # 1주당 신주배정주식수
           "EMP_BGN_DT", # 청약예정일 우리사주조합 시작일
           "EMP_END_DT", # 청약예정일 우리사주조합 종료일
           "SH_BGN_DT", # 청약예정일 구주주 시작일
           "SH_END_DT", # 청약예정일 구주주 종료일
           "PYM_DT", # 납입일
           "PRC_PLN", # 실권주 처리계획
           "LST_PLN_DT", # 신주의 상장예정일
           "CHF_AGN", # 대표주관회사
           "NST_GV_YN", # 신주인수권 양도여부
           "FIP_CM", # 확정 발행가액 산정방법
           "NST_RM", # 신주인수권에 관한 사항
           "NST_SCH_DT", # 신주인수권증서 상장예정기간
           "CI_MTH", # 증자방식
           "NOTES", # 기타 참고사항
           "RECEI_DT", # 접수날짜
           "PUB_ANN_DT", # 공시날짜
           ]
new_ic_mthn_df = pd.DataFrame(columns = columns)


save_path = "../data/doc/"

for index,n in enumerate([i for i in zip(ic_mthn_df['rcept_no'], ic_mthn_df['접수일'])]):
    
    RCEPT_NO = n[0]
    RECEI_DT = n[1]        
        
    xml_file_path = os.path.join(save_path, "%s.xml"%RCEPT_NO)
    # XML 파일 읽기
    with open(xml_file_path, "r", encoding="utf-8-sig") as xml_file:
        # XML을 딕셔너리로 변환
        xml_dict = xmltodict.parse(xml_file.read())
        
    
    ##### CST_CNT
    try:
        CST_CNT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['TABLE-GROUP'][0]['TABLE']['TBODY']['TR'][0]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['TABLE-GROUP'][0]['TABLE']['TBODY']['TR'][0]['TE']
        if "P" in TAG:
            CST_CNT = TAG['P']['#text']
            
            
    if CST_CNT != "-":
        CST_CNT = CST_CNT.replace(",","")
        CST_CNT = int(CST_CNT)


    ##### PST_CNT
    try:
        PST_CNT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['TABLE-GROUP'][0]['TABLE']['TBODY']['TR'][1]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['TABLE-GROUP'][0]['TABLE']['TBODY']['TR'][1]['TE']
        if "P" in TAG:
            PST_CNT = TAG['P']['#text']
            
    if PST_CNT != "-":
        PST_CNT = PST_CNT.replace(",","")
        PST_CNT = int(PST_CNT)
    
    
    ##### CST_ISS_VAL
    try:
        CST_ISS_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][0]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][0]['TE']
        if "P" in TAG:
            CST_ISS_VAL = TAG['P']['#text']
        elif "SPEN" in TAG:
            CST_ISS_VAL = TAG['SPEN']['#text']
    except TypeError as e:
        CST_ISS_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][0]['TE']['#text']       
    
    CST_ISS_VAL = str(CST_ISS_VAL)
    if CST_ISS_VAL != "-":
        CST_ISS_VAL = CST_ISS_VAL.replace(",","")
        CST_ISS_VAL = int(CST_ISS_VAL)
        
        
    ##### PST_ISS_VAL
    try:
        PST_ISS_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][1]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][1]['TE']
        if "P" in TAG:
            PST_ISS_VAL = TAG['P']['#text']
    except TypeError as e:
        PST_ISS_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][1]['TE']['#text']       
    
    
    if PST_ISS_VAL != "-":
        PST_ISS_VAL = PST_ISS_VAL.replace(",","")
        PST_ISS_VAL = int(PST_ISS_VAL)


    ##### PRE_CST_VAL
    try:
        PRE_CST_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][2]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][2]['TE']
        if "P" in TAG:
            PRE_CST_VAL = TAG['P']['#text']
    except TypeError as e:
        PRE_CST_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][2]['TE']['#text']       
    
    if PRE_CST_VAL != "-":
        PRE_CST_VAL = PRE_CST_VAL.replace(",","")
        PRE_CST_VAL = int(PRE_CST_VAL)    


    ##### PRE_PST_VAL
    try:
        PRE_PST_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][3]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][3]['TE']
        if "P" in TAG:
            PRE_PST_VAL = TAG['P']['#text']
    except TypeError as e:
        PRE_PST_VAL = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][3]['TE']['#text']       
    
    if PRE_PST_VAL != "-":
        PRE_PST_VAL = PRE_PST_VAL.replace(",","")
        PRE_PST_VAL = int(PRE_PST_VAL)    


    ##### PRE_CST_VAL_DT
    try:
        PRE_CST_VAL_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][2]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][2]['TU']
        if "P" in TAG:
            PRE_CST_VAL_DT = TAG['P']['#text']       
    except TypeError as e:
        PRE_CST_VAL_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][2]['TU']['#text']       
        
        
    PRE_CST_VAL_DT = PRE_CST_VAL_DT.replace("년","-")
    PRE_CST_VAL_DT = PRE_CST_VAL_DT.replace("월","-")
    PRE_CST_VAL_DT = PRE_CST_VAL_DT.replace("일","")
    PRE_CST_VAL_DT = PRE_CST_VAL_DT.replace(".","-")
    PRE_CST_VAL_DT = preprocessing_date(PRE_CST_VAL_DT)


    ##### PRE_PST_VAL_DT
    try:
        PRE_PST_VAL_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][3]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][3]['TU']
        if "P" in TAG:
            PRE_PST_VAL_DT = TAG['P']['#text']
    except TypeError as e:
        PRE_PST_VAL_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][3]['TU']['#text']       
    
    PRE_PST_VAL_DT = PRE_PST_VAL_DT.replace("년","-")
    PRE_PST_VAL_DT = PRE_PST_VAL_DT.replace("월","-")
    PRE_PST_VAL_DT = PRE_PST_VAL_DT.replace("일","")
    PRE_PST_VAL_DT = PRE_PST_VAL_DT.replace(".","-")
    PRE_PST_VAL_DT = preprocessing_date(PRE_PST_VAL_DT)
        
        
    ##### NST_GV_YN
    try:
        NST_GV_YN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][18]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][18]['TU']
        if "P" in TAG:
            NST_GV_YN = TAG['P']['#text']  
    except TypeError as e:
        NST_GV_YN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][18]['TU']['#text']       
    
    if NST_GV_YN == "예":
        NST_GV_YN = "Y"
    elif NST_GV_YN == "아니오":
        NST_GV_YN = "N"
    else:
        NST_GV_YN = "-"

    
    # ##### ISS_FUNC

    start_markers = ["신주발행가액 산정방법", "신주 발행가액 산정방법", "신주발행가액의 산정방법", "신주 발행가액 산정 방법", "신주발행가액의 산정 근거", "신주발행가액의 산정근거", "신주발행가액 산정 근거", "24. 기타 투자판단에 참고할 사항"]
    end_markers = ["나. 신주의 배정방법", "(나) 배정방법※", "다.  신주의 배정방법", "※ (확정)발행가액 산정표", "나. 청약결과 배정방법", "(2) 신주의 배정방법"]
            
    if NST_GV_YN == "Y":
        ISS_FUNC = xml_dict['DOCUMENT']['BODY']['SECTION-1']['LIBRARY']['P']
        ISS_FUNC = text_extranc(ISS_FUNC, start_markers, end_markers)
        if ISS_FUNC == "-":
            ISS_FUNC = xml_dict['DOCUMENT']['BODY']['SECTION-1']['P']
            ISS_FUNC = text_extranc(ISS_FUNC, start_markers, end_markers)
        if ISS_FUNC == "-":
            ISS_FUNC = xml_dict['DOCUMENT']['BODY']['SECTION-1']['LIBRARY']['P']
            end_markers = None
            ISS_FUNC = text_extranc(ISS_FUNC, start_markers, end_markers)
        if ISS_FUNC == "":
            ISS_FUNC = xml_dict['DOCUMENT']['BODY']['SECTION-1']['LIBRARY']['P']
            start_markers = ['나. 청약결과 배정방법']  
            end_markers = ['▶  구주주 1주당 배정비율 산출근거']    
            ISS_FUNC = text_extranc(ISS_FUNC, start_markers, end_markers)
    else:
        ISS_FUNC = "-"



    ##### ALL_BS_DT
    try:
        ALL_BS_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][5]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][5]['TU']
        if "P" in TAG:
            ALL_BS_DT = TAG['P']['#text']     
    except TypeError as e:
        ALL_BS_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][5]['TU']['#text']       

    ALL_BS_DT = ALL_BS_DT.replace("년","-")
    ALL_BS_DT = ALL_BS_DT.replace("월","-")
    ALL_BS_DT = ALL_BS_DT.replace("일","")
    ALL_BS_DT = ALL_BS_DT.replace(".","-")
    ALL_BS_DT = preprocessing_date(ALL_BS_DT)
    

    ##### NEW_ASN_CNT
    try:
        NEW_ASN_CNT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][6]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][6]['TE']
        if "P" in TAG:
            NEW_ASN_CNT = TAG['P']['#text']
        elif "SPEN" in TAG:
            NEW_ASN_CNT = TAG['SPEN']['#text']
    except TypeError as e:
        NEW_ASN_CNT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][6]['TE']['#text']       
    
    if NEW_ASN_CNT != "-":
        NEW_ASN_CNT = round(float(NEW_ASN_CNT), 3)


    ##### EMP_BGN_DT
    try:
        EMP_BGN_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][8]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][8]['TU']
        if "P" in TAG:
            EMP_BGN_DT = TAG['P']['#text']
    except TypeError as e:
        EMP_BGN_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][8]['TU']['#text']       
    
    EMP_BGN_DT = EMP_BGN_DT.replace("년","-")
    EMP_BGN_DT = EMP_BGN_DT.replace("월","-")
    EMP_BGN_DT = EMP_BGN_DT.replace("일","")
    EMP_BGN_DT = EMP_BGN_DT.replace(".","-")
    EMP_BGN_DT = preprocessing_date(EMP_BGN_DT)

    ##### EMP_END_DT
    try:
        EMP_END_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][9]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][9]['TU']
        if "P" in TAG:
            EMP_END_DT = TAG['P']['#text']
    except TypeError as e:
        EMP_END_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][9]['TU']['#text']       

    EMP_END_DT = EMP_END_DT.replace("년","-")
    EMP_END_DT = EMP_END_DT.replace("월","-")
    EMP_END_DT = EMP_END_DT.replace("일","")
    EMP_END_DT = EMP_END_DT.replace(".","-")
    EMP_END_DT = preprocessing_date(EMP_END_DT)


    ##### SH_BGN_DT
    try:
        SH_BGN_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][10]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][10]['TU']
        if "P" in TAG:
            SH_BGN_DT = TAG['P']['#text']     
    except TypeError as e:
        SH_BGN_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][10]['TU']['#text']       
    
    SH_BGN_DT = SH_BGN_DT.replace("년","-")
    SH_BGN_DT = SH_BGN_DT.replace("월","-")
    SH_BGN_DT = SH_BGN_DT.replace("일","")
    SH_BGN_DT = SH_BGN_DT.replace(".","-")
    SH_BGN_DT = preprocessing_date(SH_BGN_DT)


    ##### SH_END_DT
    try:
        SH_END_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][11]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][11]['TU']
        if "P" in TAG:
            SH_END_DT = TAG['P']['#text']   
    except TypeError as e:
        SH_END_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][11]['TU']['#text']       
    
    SH_END_DT = SH_END_DT.replace("년","-")
    SH_END_DT = SH_END_DT.replace("월","-")
    SH_END_DT = SH_END_DT.replace("일","")
    SH_END_DT = SH_END_DT.replace(".","-")
    SH_END_DT = preprocessing_date(SH_END_DT)


    ##### PYM_DT
    try:
        PYM_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][12]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][12]['TU']
        if "P" in TAG:
            PYM_DT = TAG['P']['#text']      
    except TypeError as e:
        PYM_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][12]['TU']['#text']       
    
    PYM_DT = PYM_DT.replace("년","-")
    PYM_DT = PYM_DT.replace("월","-")
    PYM_DT = PYM_DT.replace("일","")
    PYM_DT = PYM_DT.replace(".","-")
    PYM_DT = preprocessing_date(PYM_DT)
        
    ##### PRC_PLN
    try:
        PRC_PLN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][13]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][13]['TE']
        if "P" in TAG:
            PRC_PLN = TAG['P']['#text']     
    except TypeError as e:
        PRC_PLN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][13]['TE']['#text']


    start_markers = ["신주의 배정방법", "신주 발행가액 산정방법", "신주의 배정 방법", "신주배정방법", "신주발행가액의 산정방법", "기타 투자판단에 참고할 사항 참조"]
    for text in start_markers:
        if text in PRC_PLN:
            PRC_PLN = ISS_FUNC

    if PRC_PLN != "iss_func 참고":
        if "청약결과 배정방법" in PRC_PLN:
            start_markers = ["갈음할 예정입니다. ", "청약결과 배정방법 ", "다. 청약 취급처 ", "다. 청약취급처 "]
            end_markers = ["라. 신주인수권에 관한 사항", "다. 청약", "마. 기타 참고사항"]
            PRC_PLN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]["P"]
            PRC_PLN = text_extranc(PRC_PLN, start_markers, end_markers, printer=False)

            if PRC_PLN == "":
                PRC_PLN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["P"]
                PRC_PLN = text_extranc(PRC_PLN, start_markers, end_markers, printer=False)
            PRC_PLN = PRC_PLN.replace("나. 청약결과 배정방법 ", "")
            PRC_PLN = PRC_PLN.replace("다. 청약취급처", "")
            PRC_PLN = PRC_PLN.replace("다. 청약 취급처", "")


    ##### LST_PLN_DT
    try:
        LST_PLN_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][16]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][16]['TU']
        if "P" in TAG:
            LST_PLN_DT = TAG['P']['#text']       
    except TypeError as e:
        LST_PLN_DT = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][16]['TU']['#text']       

    LST_PLN_DT = LST_PLN_DT.replace("년","-")
    LST_PLN_DT = LST_PLN_DT.replace("월","-")
    LST_PLN_DT = LST_PLN_DT.replace("일","")
    LST_PLN_DT = LST_PLN_DT.replace(".","-")
    LST_PLN_DT = preprocessing_date(LST_PLN_DT)


    ##### CHF_AGN
    try:
        CHF_AGN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][17]['TE']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]['TABLE-GROUP']['TABLE']['TBODY']['TR'][17]['TE']
        if "P" in TAG:
            CHF_AGN = TAG['P']['#text']  
    except TypeError as e:
        CHF_AGN = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]['TABLE-GROUP']['TABLE']['TBODY']['TR'][17]['TE']['#text']       
    
    ##### FIP_CM
    start_markers = ["확정 발행가액 : ", "확정 발행가액: ", "확정 발행가액 산정확정 발행가액은 ", "확정 발행가액 산출근거확정 발행가액은 ", \
                    "확정 발행가액 산정 : ", "확정 발행가액 산정 확정발행가액은 ", "확정 발행가액확정 발행가액은 ", "확정 발행가액▶ ", \
                    "▶ 확정발행가액 = ", "③ ", "확정발행가액 : ", "확정 발행가액으로 합니다. ", "발행가액으로 합니다. "]
    end_markers = ["(5) 최종 발행가액", "4) 최종 발행가액", "(4) 최종 발행가액", "최종 발행가액", "1) 구주주", "발행가액을 산정하였습니다"]

    FIP_CM = retry_extraction_with_different_markers(ISS_FUNC, start_markers, end_markers)

    if FIP_CM == "-":
        end_markers = None
        FIP_CM = retry_extraction_with_different_markers(ISS_FUNC, start_markers, end_markers)


    ##### NOTES
    try:
        NOTES = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"]["P"]
    except TypeError as e:
        NOTES = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["LIBRARY"][0]["P"]

    start_markers = ["기타 참고사항", "기타 투자판단에 참고할 사항"]
    end_markers = None
    NOTES = text_extranc(NOTES, start_markers, end_markers)
    
    for ss in [":", "-"]:
        if NOTES.startswith(ss):
            NOTES = NOTES[1:].strip()


    ##### NST_RM
    # 시작 마커와 끝 마커 사이의 텍스트 추출
    start_markers = ["신주인수권증서에 관한 사항", "신주인수권에 관한 사항",]
    end_markers = ["(마) 기타 참고사항", "바.  기타 참고사항", "마. 기타 참고사항", "마. 청약이 금지되는 공매도 거래 기간"]
    try:
        NST_RM = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['LIBRARY']['P']
    except TypeError as e:
        NST_RM = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["P"]
    
    NST_RM = text_extranc(NST_RM, start_markers, end_markers, printer=False)
    if NST_RM in ["", "-"]:
        try:
            NST_RM = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['LIBRARY']['P']
        except TypeError as e:
            NST_RM = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]["P"]
        end_markers = None
        NST_RM = text_extranc(NST_RM, start_markers, end_markers, printer=False)
        for ss in [":", "-"]:
            if NST_RM.startswith(ss):
                NST_RM = NST_RM[1:].strip()
                
        if NST_RM == "":
            NST_RM = "-"


    ##### NST_SCH_DT
    if NST_RM != "-":
        start_markers = ["신주인수권증서 상장예정기간 ", "신주인수권증서 상장기간 "]
        end_markers = ["(", "4)", "⑤", "④"]
        NST_SCH_DT = retry_extraction_with_different_markers(NST_RM, start_markers, end_markers).strip()
        for ss in [":", "-"]:
            if NST_SCH_DT.startswith(ss):
                NST_SCH_DT = NST_SCH_DT[1:].strip()
        NST_SCH_DT = NST_SCH_DT.replace(" ", "")
        
        if NST_SCH_DT == "":
            NST_SCH_DT = "-"         
        
    else:
        NST_SCH_DT = "-"
        
    
    ##### CI_MTH
    try:
        CI_MTH = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['TABLE-GROUP'][0]['TABLE']['TBODY']['TR'][-1]['TU']['#text']
    except KeyError as e:
        TAG = xml_dict["DOCUMENT"]["BODY"]["SECTION-1"]['TABLE-GROUP'][0]['TABLE']['TBODY']['TR'][-1]['TU']
        if "P" in TAG:
            CI_MTH = TAG['P']['#text']  

    ##### PUB_ANN_DT
    try:
        PUB_ANN_DT = xml_dict["DOCUMENT"]["BODY"]["LIBRARY"][0]["CORRECTION"]["TABLE"][0]["TBODY"]["TR"][-1]["TD"]["#text"]
    except KeyError as e:
        PUB_ANN_DT = xml_dict["DOCUMENT"]["BODY"]["LIBRARY"]["SECTION-1"]["TABLE"]["TBODY"]["TR"][2]["TD"][-1]["#text"]
    
    PUB_ANN_DT = PUB_ANN_DT.replace("년","-")
    PUB_ANN_DT = PUB_ANN_DT.replace("월","-")
    PUB_ANN_DT = PUB_ANN_DT.replace("일","")
    PUB_ANN_DT = PUB_ANN_DT.replace(" ","")
    PUB_ANN_DT = preprocessing_date(PUB_ANN_DT)

    new_ic_mthn_df.at[index, "RCEPT_NO"] = RCEPT_NO
    new_ic_mthn_df.at[index, "CST_CNT"] = CST_CNT
    new_ic_mthn_df.at[index, "PST_CNT"] = PST_CNT
    new_ic_mthn_df.at[index, "CST_ISS_VAL"] = CST_ISS_VAL
    new_ic_mthn_df.at[index, "PST_ISS_VAL"] = PST_ISS_VAL
    new_ic_mthn_df.at[index, "PRE_CST_VAL"] = PRE_CST_VAL
    new_ic_mthn_df.at[index, "PRE_PST_VAL"] = PRE_PST_VAL
    new_ic_mthn_df.at[index, "PRE_CST_VAL_DT"] = PRE_CST_VAL_DT
    new_ic_mthn_df.at[index, "PRE_PST_VAL_DT"] = PRE_PST_VAL_DT
    new_ic_mthn_df.at[index, "ISS_FUNC"] = ISS_FUNC
    new_ic_mthn_df.at[index, "ALL_BS_DT"] = ALL_BS_DT
    new_ic_mthn_df.at[index, "NEW_ASN_CNT"] = NEW_ASN_CNT
    new_ic_mthn_df.at[index, "EMP_BGN_DT"] = EMP_BGN_DT
    new_ic_mthn_df.at[index, "EMP_END_DT"] = EMP_END_DT
    new_ic_mthn_df.at[index, "SH_BGN_DT"] = SH_BGN_DT
    new_ic_mthn_df.at[index, "SH_END_DT"] = SH_END_DT
    new_ic_mthn_df.at[index, "PYM_DT"] = PYM_DT
    new_ic_mthn_df.at[index, "PRC_PLN"] = PRC_PLN
    new_ic_mthn_df.at[index, "LST_PLN_DT"] = LST_PLN_DT
    new_ic_mthn_df.at[index, "CHF_AGN"] = CHF_AGN
    new_ic_mthn_df.at[index, "NST_GV_YN"] = NST_GV_YN
    new_ic_mthn_df.at[index, "FIP_CM"] = FIP_CM
    new_ic_mthn_df.at[index, "NST_RM"] = NST_RM
    new_ic_mthn_df.at[index, "NST_SCH_DT"] = NST_SCH_DT
    new_ic_mthn_df.at[index, "CI_MTH"] = CI_MTH
    new_ic_mthn_df.at[index, "NOTES"] = NOTES
    new_ic_mthn_df.at[index, "RECEI_DT"] = RECEI_DT
    new_ic_mthn_df.at[index, "PUB_ANN_DT"] = PUB_ANN_DT

new_ic_mthn_df.index.name = "ID"

In [11]:
path = "../data/juju_data_add_20150101_20230919.csv"
new_ic_mthn_df.to_csv(path, encoding="utf-8-sig")